# Basic model introduction

This page introduces the processes for building and running a simple compartmental disease model with _summer_.
In the following example, we will create an SIR compartmental model for a general, unspecified emerging infectious disease spreading through a fully susceptible population.
The objective of this notebook is to introduce the _summer_ interface that we will use throughout this series of notebooks,
and to use it to demonstrate the construction of a simple model.
We will then use this simple model to demonstrate some basic principles of infectious disease modelling,
although there are several principles and model features that we will come back to in subsequent notebooks.

In this model there will be:

- Three compartments: susceptible, infected and recovered
- A starting population of 1000 people, with 10 of them infected (and infectious)
- An evaluation timespan from day zero to 20 units of time
- Inter-compartmental flows for infection, deaths and recovery

Let's look at a complete example of this model in action, and then examine the details of each step.
This is the complete example model that we will be working with.
As with any Python script, we first have to import the objects we need,
plus we'll set the visualisation of pandas objects to be interactive.

In [ ]:
# If we are running in google colab, pip install the required packages, 
# but do not modify local environments
try:
  import google.colab
  IN_COLAB = True
  %pip install summerepi
except:
  IN_COLAB = False

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict

from summer import CompartmentalModel
from summer.parameters import Parameter as param

pd.options.plotting.backend = "plotly"

First, let's create a function that gives us a base SIR model with the
basic compartments, starting populations and inter-compartmental flows implemented.

In [ ]:
def get_base_sir_model() -> CompartmentalModel:
    """
    Generate an instance of an SIR model with some standard parameters, 
    population distribution and parameters hard-coded.
    
    Returns:
        The summer model object
    """
    compartments = (
        "susceptible",
        "infectious",
        "recovered",
    )
    analysis_times = (0, 20)
    
    model = CompartmentalModel(
        times=analysis_times,
        compartments=compartments,
        infectious_compartments=["infectious"],
    )
    model.set_initial_population(
        distribution={"susceptible": 990, "infectious": 10}
    )
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=1.,
        source="susceptible", 
        dest="infectious",
    )
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=0.333,
        source="infectious", 
        dest="recovered",
    )
    model.add_death_flow(
        name="infection_death", 
        death_rate=0.05,
        source="infectious",
    )
    return model

## Building the model
In this simple example model, we have written instructions for how to create a compartmental model object that consists of three compartments.
We have then provided a period of time over which we will analyse the model.
(We can think of these as days, but the time unit is arbitrary as far as the model is concerned.)
We have then identified the compartment named "infectious" as the compartment that will contribute to the force of infection.
Next we have distributed the starting popuation across two of the compartments
(with the unnamed compartment, "recovered", not receiving any starting population).
Next we have introduced an infection frequency flow that transitions people from the susceptible to the infectious compartment.
Last, we have introduced some fixed flow rates that transition people from the infectious compartment to the recovered compartment,
and a fixed transition flow that removes people from the model to represent death.
The specifics of how these flows work will be discussed in notebook 3.

## Using the function to build the model
Then use the function to get an instance of this model,
run it and have a look at the compartment size progression over time.
Note that we use the plotting functions built-in to pandas objects to do this.
Pandas is a very widely used library for data processing, which we will use extensively in this series.
Because there may be considerable data wrangling necessary for our model outputs after we have run our model,
we prefer use external libraries for this sort of processing.
This is because these analysis stages have very few features that is specific to infectious diseases modelling,
such that it is preferable to use well-developed external libraries.
Libraries like pandas have several advantages, including extensive code documentation,
availability of online help (e.g. through forums such as Stack Overflow),
and methods to support data processing.
Here we use pandas easy integration with _plotly_ to create an interactive plot from the pandas
object in a single line of code.
(Note that this could easily be set to other output formats,
e.g. by setting the pandas plotting backend to _matplotlib_ above.)

In [ ]:
base_model = get_base_sir_model()  # Get the model object
base_model.run()  # Run it
compartment_values = base_model.get_outputs_df()  # Access the outputs as a pandas dataframe
compartment_values.plot()  # Plot

However, we can do this slightly better,
by asking the model object to **_expect_** certain parameters,
which we define as parameter objects,
but don't give values yet.
We'll come back to why this is worth doing later,
but for now let's just say it is the preferred approach.
While we're at it, we'll also make the times and starting
population values part of the inputs to this function,
even though they go into the model object just as values.

In [ ]:
def get_param_aware_sir_model(
    parameters: Dict,
) -> CompartmentalModel:
    """
    Generate an instance of an SIR model that is expecting parameter values to be provided
    for the transition rates and some other features.
    
    Args:
        parameters: The parameter values to be used in running the model
    Returns:
        The summer model object
    """
    
    # Define the 
    compartments = (
        "susceptible",
        "infectious",
        "recovered",
    )
    infectious_compartment = [
        "infectious",
    ]
    analysis_times = (
        parameters["start_time"], 
        parameters["end_time"]
    )

    model = CompartmentalModel(
        times=analysis_times,
        compartments=compartments,
        infectious_compartments=infectious_compartment,
        takes_params=True,
    )
    
    # Check and assign infectious seed
    pop = parameters["population"]
    seed = parameters["seed"]
    suscept_pop = pop - seed
    msg = "Seed larger than population"
    assert pop >= 0., msg
    
    model.set_initial_population(
        distribution={
            "susceptible": suscept_pop, 
            "infectious": seed}
    )
    
    # Add a frequency-dependent transmission flow
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=param("contact_rate"), 
        source="susceptible", 
        dest="infectious",
    )
    
    # Add a constant recovery flow
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=param("recovery_rate"), 
        source="infectious", 
        dest="recovered",
    )
    
    # Add a constant infection-related death rate
    model.add_death_flow(
        name="infection_death", 
        death_rate=param("death_rate"), 
        source="infectious",
    )
    return model

Now let's use that function, specifying our parameters all in one go
and feeding them into the model object when we're ready to run it.

In [ ]:
parameters = {
    "contact_rate": 1.,
    "recovery_rate": 0.333,
    "death_rate": 0.05,
    "population": 1000.,
    "seed": 10.,
    "start_time": 0.,
    "end_time": 20.,
}

param_aware_model = get_param_aware_sir_model(parameters)
param_aware_model.run(parameters=parameters)
compartment_values = param_aware_model.get_outputs_df()
compartment_values.plot()

## Digging into the model object
Now that we have our [`CompartmentalModel`](/api/model.html) object,
we can use this structure to inspect some aspects of what is going on under the surface,
for example, compartments, flows and others attributes.
This is **_highly recommended_**, 
to ensure that the model you have created is consistent with what you were wanting.
Try out using tab complete in this notebook to inspect the range of methods and
attributes that are available for a `CompartmentalModel` object.

In [ ]:
print(param_aware_model.compartments)
print(param_aware_model.initial_population)
print(param_aware_model.times)
print(compartment_values)

## Epidemiological messages
This is clearly a very simple model of an epidemic caused by a short-lived pathogen that induces complete immunity in its host.
However, despite its simplicity, it does capture a surprising number of the actual features of an epidemic caused by an infection
of this type.
In general, models of infectious diseases transmission should be as complicated as they need to be,
which means that the additional complexity that we might need to inject into this model is highly dependent on the purpose that
we will be using it for - or the epidemiological question that we will be addressing through our analysis.
It may also be dependent on us having sufficient epidemiological understanding of the epidemic to be able to incorporate these
features with a reasonable level of confidence that we are actually capturing the processes that we are interested in
(including empiric data to estimate the parameters that we need to build our more complicated model).

Let's think of some of the epidemiological features that this very simple model **_does_** capture:
- Very broadly, this model does give us the shape that epi curves often follow - looking vaguely like a bell
- There is an exponential growth phase when the population remains largely susceptible
- The growth in the epidemic decreases as the proportion of susceptibles decreases
- As the effective reproduction number falls past one, the epidemic peaks and begins to decline
- The epidemic dies out as the susceptibles decrease and the effective reproduction number drops below one
- As the epidemic ends and transmission declines to approximately zero, susceptibles are depleted, but not completely - a proportion of the population remains susceptible even after the epidemic

Let's think of some of the epidemiological features that this model **_does not_** capture:
- Any heterogeneity in the background population with regards progression through infection states after exposure
- Any heterogeneity in transmission, such as greater transmission to people within the population with similar characteristics
- Any heterogeneity in the pathogen, such as multiple strains with different characteristics circulating through the population
- Any changes in how people transition through their stages over time that might be induced through changes external to the model
(i.e. other than those related to the changes in the population distribution across compartments resulting from transmission of this immunising pathogen)
- Tracking any outputs other than the sizes of the model's compartments

We will return to these features and how to elaborate our base model to capture them over the following notebooks.

## Summary

That's it for now, now you know how to:

- Create a model
- Assing the starting population
- The syntax for adding flows (although we'll come back to what these mean)
- Run the model
- Access and visualise the outputs

A detailed API reference for the `CompartmentalModel` class can be found [here](http://summerepi.com/api/model.html)